In [102]:
import pandas as pd
import os

In [103]:
from sqlalchemy.engine import create_engine

engine = create_engine('postgresql://localhost/mimic') 

In [104]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [105]:
sepsis3 = pd.read_csv('~/Downloads/sepsis3.csv')

In [106]:
os.environ['DATABASE_URL'] = 'postgresql://localhost/mimic'

In [107]:
%%sql
SET search_path TO mimiciii;

 * postgresql://localhost/mimic
Done.


[]

In [108]:
%%sql total_cohort << 

WITH patientcohort AS (
  select 
    t1.hadm_id, 
    t1.ethnicity, 
    t1.admittime, 
    (
      extract(
        year 
        from 
          t1.admittime
      ) - extract(
        year 
        from 
          t2.dob
      )
    ) as age_yr, 
    t2.gender, 
    t2.dob, 
    t2.dod, 
    t2.dod_hosp, 
    t2.dod_ssn, 
    t2.expire_flag 
  from 
    admissions t1 
    left join patients t2 on t1.subject_id = t2.subject_id
), 
oasisscores as (
  select 
    icustay_id, 
    oasis 
  from 
    oasis
), 
vasop_1 as (
  SELECT vaso.icustay_id, MIN(starttime) as vaso_start, MAX(CASE WHEN endtime > intime + interval '24 hours' THEN intime + interval '24 hours' ELSE endtime END) as vaso_end, SUM(amount) as max_vaso_dosage
  FROM VASOPRESSORDURATIONS_WO_MILI_DOBU vaso
  LEFT JOIN icustays ic ON ic.icustay_id = vaso.icustay_id
  WHERE intime + interval '24 hours' > starttime
  GROUP BY vaso.icustay_id
),
vasop_2 as (
    SELECT icustay_id, extract(epoch from vaso_end - vaso_start)/(60*60.0) as vaso_duration, max_vaso_dosage
    FROM vasop_1
),
heightweight as (
  select 
    icustay_id, 
    coalesce(
      height_first, height_max, height_min
    ) as height, 
    coalesce(
      weight_first, weight_max, weight_min
    ) as weight 
  from 
    heightweight
), 
icustays as(
  select 
    t2.HADM_ID, 
    t2.ICUSTAY_ID, 
    t2.INTIME, 
    t2.OUTTIME, 
    t2.first_careunit, 
    t2.LOS
  from 
    admissions t1 
    left join icustays t2 on t1.HADM_ID = t2.HADM_ID
), 
firstdaylabs as (
  select 
    icustay_id, 
    creatinine_max, 
    creatinine_min, 
    bicarbonate_min, 
    bicarbonate_max, 
    lactate_min, 
    lactate_max, 
    potassium_min, 
    potassium_max, 
    bun_min, 
    bun_max, 
    wbc_min, 
    wbc_max 
  from 
    labsfirstday
), 
bloodgasfday as (
  select 
    icustay_id, 
    min(ph) as ph 
  from 
    bloodgasfirstday 
  group by 
    icustay_id
), 
sofa as (
  select 
    icustay_id, 
    sofa 
  from 
    sofa
), 
sapsii as (
  select 
    icustay_id, 
    sapsii 
  from 
    sapsii
), 
all_patient as (
  select 
    p.SUBJECT_ID, 
    p.gender, 
    p.dob, 
    p.dod, 
    p.dod_hosp, 
    p.expire_flag 
  from 
    patients p
), 
cleaned_admission as (
  select 
    a.hadm_id, 
    SUBJECT_ID, 
    a.ethnicity, 
    a.admittime, 
    DISCHTIME, 
    ADMISSION_TYPE, 
    DEATHTIME, 
    DIAGNOSIS, 
    HOSPITAL_EXPIRE_FLAG,
    admission_location
  from 
    admissions a
),
kdigo_stages_max_1 as (
    SELECT kd.icustay_id, max(aki_stage) as max_aki_score
    FROM kdigo_stages kd
    LEFT JOIN icustays ic ON ic.icustay_id = kd.icustay_id
    WHERE
        charttime < intime + interval '24 hours'
    group by kd.icustay_id
),  
kdigo_stages_max as (
    SELECT mo.icustay_id, min(max_aki_score) as max, min(charttime) as time
    FROM kdigo_stages_max_1 mo
    LEFT JOIN kdigo_stages kd ON mo.icustay_id = kd.icustay_id
    WHERE kd.aki_stage = max_aki_score
    GROUP BY mo.icustay_id
),
was_ventilated_24h as (
 SELECT vent.icustay_id, CASE WHEN min(extract(epoch from starttime)) <= extract(epoch from intime) + 24*60*60 THEN 1 ELSE 0 END as was_ventilated_24h
 FROM ventdurations vent
    LEFT JOIN icustays ic ON ic.icustay_id = vent.icustay_id
 GROUP BY vent.icustay_id, ic.intime
),
wtdiff as (
  with wtmaxmin as (
    select 
      icustay_id, 
      max(weight) as wt_max, 
      min(weight) as wt_min 
    from 
      weightdurations 
    group by 
      icustay_id
  ), 
  wtdayone as (
    select 
      icustay_id, 
      coalesce(weight_admit, weight) as wt_dayone 
    from 
      weightfirstday
  ) 
  select 
    t1.icustay_id, 
    t1.wt_max, 
    t1.wt_min, 
    t2.wt_dayone, 
    (t1.wt_max - t2.wt_dayone) as wtdifft0, 
    (t1.wt_max - t1.wt_min) as wtdiffmaxmin 
  from 
    wtmaxmin t1 
    join wtdayone t2 on t1.icustay_id = t2.icustay_id 
  where 
    (t1.wt_max - t2.wt_dayone)>= 0.01
) 
SELECT 
  p.SUBJECT_ID, 
  icustays.ICUSTAY_ID, 
  p.gender, 
  p.dob, 
  age_yr, 
  p.dod, 
  p.dod_hosp, 
  p.expire_flag, 
  a.hadm_id, 
  a.ADMISSION_TYPE, 
  a.ethnicity, 
  a.admittime, 
  a.DISCHTIME, 
  a.DEATHTIME, 
  a.DIAGNOSIS, 
  a.HOSPITAL_EXPIRE_FLAG, 
  a.admission_location,
  icustays.INTIME, 
  icustays.OUTTIME, 
  icustays.LOS, 
  icustays.first_careunit, 
  oasisscores.oasis, 
  heightweight.height, 
  heightweight.weight, 
HeartRate_Min
,HeartRate_Max
,HeartRate_Mean
,SysBP_Min
,SysBP_Max
,SysBP_Mean
,DiasBP_Min
,DiasBP_Max
,DiasBP_Mean
,MeanBP_Min
,MeanBP_Max
,MeanBP_Mean
,RespRate_Min
,RespRate_Max
,RespRate_Mean
,TempC_Min
,TempC_Max
,TempC_Mean
,SpO2_Min
,SpO2_Max
,SpO2_Mean
,Glucose_Min
,Glucose_Max
,Glucose_Mean,
  ph as min_ph_icustay, 
  sapsii.sapsii as sapsii_score, 
  sofa.sofa as sofa_score, 
  (extract(epoch from icustays.intime) - extract(epoch from a.admittime)) / (60*60*24) as los_preicu, 
  wtdifft0 as wtdiff,
  kdigo_stages_max.max as max_kdigo,
  kdigo_stages_max.time as max_kdigo_time,
  was_ventilated_24h.was_ventilated_24h,
vasop_2.vaso_duration, 
vasop_2.max_vaso_dosage
from 
  all_patient p 
  right join cleaned_admission a on p.SUBJECT_ID = a.SUBJECT_ID 
  join icustays on icustays.HADM_ID = a.HADM_ID 
  left join oasisscores on icustays.icustay_id = oasisscores.icustay_id 
  left join heightweight on icustays.icustay_id = heightweight.icustay_id 
  left join firstdaylabs on icustays.icustay_id = firstdaylabs.icustay_id 
  left join bloodgasfday on icustays.icustay_id = bloodgasfday.icustay_id 
  left join sofa on icustays.icustay_id = sofa.icustay_id 
  left join sapsii on icustays.icustay_id = sapsii.icustay_id 
  left join patientcohort on icustays.hadm_id = patientcohort.hadm_id 
  left join wtdiff on icustays.icustay_id = wtdiff.icustay_id
  left join vitalsfirstday on vitalsfirstday.icustay_id = icustays.icustay_id
  left join kdigo_stages_max on kdigo_stages_max.icustay_id = icustays.icustay_id
  left join was_ventilated_24h ON was_ventilated_24h.icustay_id = icustays.icustay_id
  left join vasop_2 ON vasop_2.icustay_id = icustays.icustay_id

 * postgresql://localhost/mimic
61532 rows affected.
Returning data to local variable total_cohort


In [109]:
%%sql comorb_indicators <<
SELECT *
FROM ELIXHAUSER_QUAN

 * postgresql://localhost/mimic
58976 rows affected.
Returning data to local variable comorb_indicators


In [110]:
comorb_indicators = comorb_indicators.DataFrame()

In [111]:
%%sql crrt <<
SELECT crrt.icustay_id, duration_hours as crrt_durations
FROM crrtdurations crrt
LEFT JOIN icustays ic on crrt.icustay_id = ic.icustay_id
WHERE starttime > ic.intime + interval '24 hours'

 * postgresql://localhost/mimic
5526 rows affected.
Returning data to local variable crrt


In [112]:
%%sql rrt <<
SELECT rrt.icustay_id, MAX(1) as rrt
FROM pivoted_rrt rrt
LEFT JOIN icustays ic ON ic.icustay_id = rrt.icustay_id
WHERE
	rrt.charttime >= ic.intime + interval '24 hours'
	AND rrt.dialysis_active = 1
	AND rrt.dialysis_type <> 'Peritoneal'
GROUP BY rrt.icustay_id

 * postgresql://localhost/mimic
1446 rows affected.
Returning data to local variable rrt


In [113]:
rrt = rrt.DataFrame()

In [114]:
%%sql icustays <<
SELECT *
FROM icustays

 * postgresql://localhost/mimic
61532 rows affected.
Returning data to local variable icustays


In [115]:
%%sql esrd << 
SELECT hadm_id, 1 as esrd FROM diagnoses_icd di WHERE icd9_code = '5856' 

 * postgresql://localhost/mimic
1926 rows affected.
Returning data to local variable esrd


In [116]:
icustays = icustays.DataFrame()
icustays = icustays[icustays.dbsource == 'metavision']
icustays.head()

,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
25,390,291,125726,275109,metavision,CCU,CCU,7,7,2106-04-17 12:26:17,2106-04-18 22:05:39,1.4023
26,391,291,126219,246725,metavision,MICU,MICU,52,52,2107-09-13 22:43:01,2107-09-14 18:34:48,0.8276
45,410,305,194340,217232,metavision,SICU,SICU,33,33,2129-09-03 12:31:31,2129-09-05 23:00:50,2.4370
130,90,85,112077,291697,metavision,MICU,MICU,23,23,2167-07-25 18:50:37,2167-07-27 18:37:35,1.9909
156,116,107,182383,252542,metavision,MICU,MICU,23,23,2121-11-30 19:24:56,2121-12-01 21:20:57,1.0806


In [117]:
icustays = icustays.merge(sepsis3, on='icustay_id', how='left')
icustays.loc[icustays.sepsis3.isna(), 'sepsis3'] = 0

In [118]:
icustays = icustays.merge(esrd.DataFrame(), on='hadm_id', how='left')
icustays.loc[icustays.esrd.isna(), 'esrd'] = 0

In [119]:
crrt = crrt.DataFrame().groupby('icustay_id').agg({'crrt_durations' : 'max'}).reset_index()
icustays = icustays.merge(crrt, on='icustay_id', how='left')
icustays.loc[icustays.crrt_durations > 0.0, 'crrt'] = 1
icustays.loc[icustays.crrt_durations == 0.0, 'crrt'] = 0

In [120]:
icustays = icustays.merge(rrt, on='icustay_id', how='left')
icustays.loc[icustays.rrt.isna(), 'rrt'] = 0

In [121]:
icustays = icustays.merge(total_cohort.DataFrame().drop(columns=['hadm_id', 'first_careunit', 'subject_id', 'los', 'intime', 'outtime']), on='icustay_id', how='left')
icustays.loc[:, 'got_vasopressors'] = icustays['vaso_duration'] > 0
icustays['got_vasopressors'] = icustays['got_vasopressors'].astype(int)


In [122]:
icustays = icustays.merge(comorb_indicators, on='hadm_id', how='left')

features_comorb = [_ for _ in comorb_indicators.columns if not _ in ['icustay_id']]
for _ in features_comorb:
    icustays.loc[icustays[_].isna(), _] = 0

In [123]:
sorted(icustays.columns)

[u'admission_location',
 u'admission_type',
 u'admittime',
 u'age_yr',
 u'aids',
 u'alcohol_abuse',
 u'blood_loss_anemia',
 u'cardiac_arrhythmias',
 u'chronic_pulmonary',
 u'coagulopathy',
 u'congestive_heart_failure',
 'crrt',
 'crrt_durations',
 u'dbsource',
 u'deathtime',
 u'deficiency_anemias',
 u'depression',
 u'diabetes_complicated',
 u'diabetes_uncomplicated',
 u'diagnosis',
 u'diasbp_max',
 u'diasbp_mean',
 u'diasbp_min',
 u'dischtime',
 u'dob',
 u'dod',
 u'dod_hosp',
 u'drug_abuse',
 u'esrd',
 u'ethnicity',
 u'expire_flag',
 u'first_careunit',
 u'first_wardid',
 u'fluid_electrolyte',
 u'gender',
 u'glucose_max',
 u'glucose_mean',
 u'glucose_min',
 'got_vasopressors',
 u'hadm_id',
 u'heartrate_max',
 u'heartrate_mean',
 u'heartrate_min',
 u'height',
 u'hospital_expire_flag',
 u'hypertension',
 u'hypothyroidism',
 u'icustay_id',
 u'intime',
 u'last_careunit',
 u'last_wardid',
 u'liver_disease',
 u'los',
 u'los_preicu',
 u'lymphoma',
 u'max_kdigo',
 u'max_kdigo_time',
 u'max_vaso

In [124]:
icustays.icustay_id.nunique()

23620

In [125]:
icustays[(icustays.age_yr >= 18) & (icustays.sepsis3 == 1)].icustay_id.nunique()

14233

In [126]:
icustays[(icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & ((icustays.los >= 2.0))].icustay_id.nunique()

8527

In [127]:
icustays[(icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & (icustays.esrd == 0) & ((icustays.los >= 2.0))].icustay_id.nunique()

7892

In [128]:
icustays[(icustays.max_kdigo > 0) & (icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & (icustays.esrd == 0) & (icustays.los >= 2.0)].icustay_id.nunique()

4754

In [129]:
icustays[(icustays.max_kdigo > 0) & (icustays.rrt == 1.0) & (icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & (icustays.esrd == 0) & (icustays.los >= 2.0)].icustay_id.nunique()

386

In [130]:
icustays[(icustays.max_kdigo > 0) & (icustays.rrt == 0) & (icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & (icustays.esrd == 0) & (icustays.los >= 2.0)].icustay_id.nunique()

4368

In [131]:
icustays[(icustays.max_kdigo > 0) & (icustays.crrt == 1) & (icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & (icustays.esrd == 0) & (icustays.los >= 2.0)].icustay_id.nunique()

280

In [137]:
icustays[(icustays.max_kdigo > 0) & (icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & (icustays.esrd == 0) & (icustays.los >= 2.0)].to_csv('./cohort_aki_sepsis3_11clock16.csv')

In [136]:
icustays[(icustays.rrt == 1) & (icustays.max_kdigo > 0) & (icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & (icustays.esrd == 0) & (icustays.los >= 2.0)].shape

(386, 99)

In [138]:
icustays[(icustays.max_kdigo > 0) & (icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & (icustays.esrd == 0) & (icustays.los >= 2.0)].shape

(4754, 99)

In [ ]:
 icustays[(icustays.max_kdigo > 0) & (icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & (icustays.esrd == 0) & (icustays.los >= 2.0)]

In [133]:
icustays.shape

(23620, 99)

In [134]:
icustays[(icustays.age_yr >= 18) & (icustays.sepsis3 == 1) & (icustays.esrd == 0) & (icustays.los >= 2.0)].shape

(7892, 99)

In [ ]:
print(classification_report(y_true, y_pred, target_names=target_names))

## Table 1

### Backup

In [131]:
vasopressors = vasopressors[~((vasopressors.endtime < vasopressors.intime) | (vasopressors.starttime > vasopressors.outtime))].reset_index(drop=True)

In [134]:
antibiotics = pd.read_csv('~/Downloads/antibiotics_no_limit.csv')
antibiotics.antibiotic_time = pd.to_datetime(antibiotics.antibiotic_time)
antibiotics.antibiotic_endtime = pd.to_datetime(antibiotics.antibiotic_endtime)

,hadm_id,icustay_id,antibiotic_time,antibiotic_endtime
0,116009,216609,2113-02-15 00:20:44,2113-02-16 00:00:00
1,116009,216609,2113-02-15 00:20:44,2113-02-16 00:00:00
2,116009,216609,2113-02-15 00:20:44,2113-02-17 00:00:00
3,115385,256068,2148-11-12 00:00:00,2148-11-12 16:46:47
4,156857,239612,2163-09-23 00:00:00,2163-09-23 00:00:00


In [135]:
antibiotics = antibiotics.dropna(subset=['antibiotic_time']).reset_index(drop=True)

In [138]:
merged = vasopressors.merge(antibiotics, on='icustay_id', how='left').copy()

,icustay_id,vasonum,starttime,endtime,duration_hours,intime,outtime,hadm_id,antibiotic_time,antibiotic_endtime
0,200024,1,2127-03-03 16:15:00,2127-03-03 20:35:00,4.333333,2127-03-03 16:09:07,2127-03-04 01:18:06,NaN,NaT,NaT
1,200028,1,2133-10-29 17:47:00,2133-10-30 14:31:00,20.733333,2133-10-29 17:13:50,2133-11-01 14:55:14,NaN,NaT,NaT
2,200033,1,2198-08-10 18:30:00,2198-08-10 19:45:00,1.250000,2198-08-07 17:56:17,2198-08-21 14:59:18,198650.0,2198-08-14,2198-08-15
3,200033,1,2198-08-10 18:30:00,2198-08-10 19:45:00,1.250000,2198-08-07 17:56:17,2198-08-21 14:59:18,198650.0,2198-08-14,2198-08-20
4,200033,1,2198-08-10 18:30:00,2198-08-10 19:45:00,1.250000,2198-08-07 17:56:17,2198-08-21 14:59:18,198650.0,2198-08-14,2198-08-20


In [142]:
merged = merged.dropna(subset=['antibiotic_time']).reset_index(drop=True).copy()

In [143]:
merged_2 = merged[(((merged.antibiotic_time <= merged.starttime) & (merged.antibiotic_endtime >= merged.starttime)) | ((merged.antibiotic_time <= merged.endtime) & (merged.antibiotic_endtime >= merged.endtime)))].copy().reset_index(drop=True)

In [147]:
pd.DataFrame([[_] for _ in list(merged_2.icustay_id.unique())], columns=['hadm_id']).to_csv('final_hadm_ids.csv')